In [11]:
import numpy as np
from sklearn.ensemble import AdaBoostRegressor
import pandas as pd
from sklearn.metrics import make_scorer, mean_squared_log_error, mean_absolute_error, mean_absolute_percentage_error,

mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor, BaggingRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score
# from ray.tune.sklearn import TuneSearchCV
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
import torch
from ray import tune
from scipy.stats import randint, loguniform

np.random.seed(42)
all_features, y = torch.load('data/house_price_v2.pkl')
X = all_features.iloc[:len(y), :]
X, y = X, y.values
X_test = all_features.iloc[len(y):, :]

model = AdaBoostRegressor(base_estimator=SVR())
param = {
    'learning_rate': loguniform(1e-1, 1),
    'base_estimator__C': loguniform(1e-3, 1e3),
    'base_estimator__gamma': loguniform(1e-6, 10),
    'n_estimators': randint(10, 500)
}

In [17]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

optim = HalvingRandomSearchCV(model, param, cv=10, n_jobs=-1, verbose=1,
                              scoring=make_scorer(mean_squared_error, greater_is_better=False))
optim.fit(X, y)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 20
max_resources_: 1460
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 73
n_resources: 20
Fitting 10 folds for each of 73 candidates, totalling 730 fits
----------
iter: 1
n_candidates: 25
n_resources: 60
Fitting 10 folds for each of 25 candidates, totalling 250 fits
----------
iter: 2
n_candidates: 9
n_resources: 180
Fitting 10 folds for each of 9 candidates, totalling 90 fits
----------
iter: 3
n_candidates: 3
n_resources: 540
Fitting 10 folds for each of 3 candidates, totalling 30 fits


HalvingRandomSearchCV(cv=10, estimator=AdaBoostRegressor(base_estimator=SVR()),
                      n_jobs=-1,
                      param_distributions={'base_estimator__C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001BBA3A07AC0>,
                                           'base_estimator__gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001BBA3C622E0>,
                                           'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001BBA19CA2E0>,
                                           'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001BBA3BA4700>},
                      scoring=make_scorer(mean_squared_error, greater_is_better=False),
                      verbose=1)

In [22]:
print(optim.best_params_)

{'base_estimator__C': 10.879585646155862, 'base_estimator__gamma': 0.000506547522932075, 'learning_rate': 0.31729633401624546, 'n_estimators': 295}


In [19]:
best_model = optim.best_estimator_
mean_squared_error(best_model.predict(X), y)

0.005251549910547243

In [23]:
import pandas as pd

y_test = best_model.predict(X_test.values)
result = pd.DataFrame(np.exp(y_test), index=X_test.index, columns=['SalePrice'], )
result.to_csv('Adaboost_SVM.csv')

d:\project\python\.venv\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but AdaBoostRegressor was fitted with feature names
  warnings.warn(
